# Overview
This script builds a text prediction model using LSTM (Long Short-Term Memory) and Word2Vec embeddings. The model is trained on a dataset from the PersonaHub collection, and it predicts the next word in a sequence based on a given input sentence.

Installation
Before running the code, install the required packages using:

# **Installation**
Before running the code, install the required packages using

In [ ]:
!pip install datasets
!pip install nltk datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

# Imports
The script imports the following libraries:

gensim: For loading pre-trained Word2Vec models.
tensorflow.keras: For building and training the LSTM model.
nltk: For text preprocessing and tokenization.
datasets: For loading the dataset.
numpy: For numerical operations.
re: For regular expression operations.

In [ ]:
import gensim.downloader as api
import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,SimpleRNN
from nltk.tokenize import word_tokenize
import re
from datasets import load_dataset
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import gensim.downloader as api
import numpy as np
import tensorflow as tf

import nltk
from nltk.corpus import stopwords


# Dataset Loading


In [ ]:
from datasets import load_dataset

ds = load_dataset("proj-persona/PersonaHub", "instruction")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

# Text Preprocessing
 The preprocess_text function processes the text as follows:

Converts text to lowercase.

1. Converts text to lowercase.

2.  Removes special characters, URLs, and mentions.

3. Tokenizes the text into words..





In [ ]:
# Load dataset (assuming it is loaded as 'dataset')
# Example: dataset = load_dataset('your_dataset_path')

nltk.download('punkt')
nltk.download('stopwords')
# Initialize stop words
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    words = word_tokenize(text)
    return ' '.join(words)

# Preprocessing and Tokenization
all_text = ''

for row in ds['train']:
    text = row['synthesized text']
    processed_text = preprocess_text(text)
    all_text += ' ' + processed_text

tokenizer = Tokenizer()
tokenizer.fit_on_texts([all_text])
sequences = tokenizer.texts_to_sequences([all_text])[0]
vocab_size = len(tokenizer.word_index) + 1

# Preparing Data
input_sequences = []
output_sequences = []
sequence_length = 5  # Number of words to consider as input for prediction

for i in range(len(sequences) - sequence_length):
    input_sequences.append(sequences[i:i + sequence_length])
    output_sequences.append(sequences[i + sequence_length])

input_sequences = np.array(input_sequences)
output_sequences = np.array(output_sequences)

print(f'Input Sequences Shape: {input_sequences.shape}')
print(f'Output Sequences Shape: {output_sequences.shape}')
print(f'Vocabulary Size: {vocab_size}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Input Sequences Shape: (2068589, 5)
Output Sequences Shape: (2068589,)
Vocabulary Size: 68888


In [ ]:
output_sequences.shape

(2068589,)

In [ ]:
input_sequences.shape

(2068589, 5)

# Isntaling word2vec model

In [ ]:
# Word2vec Model
word2vec_model = api.load("word2vec-google-news-300")

#word2vec_model =api.load("glove-twitter-25")


[==================================================] 100.0% 1662.8/1662.8MB downloaded


# Embedding Matrix
Creates an embedding matrix using Word2Vec embeddings:


*   Each word's vector is stored in the matrix if it exists in the pre-trained model

In [ ]:
# Creating Embedding Matrix
embedding_dim = word2vec_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, index in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[index] = word2vec_model[word]

# LSTM Model

* Embedding Layer: Uses pre-trained Word2Vec embeddings.

*   LSTM Layer: Processes sequences of words.

*  Dense Layer: Outputs predictions for the next word.






In [ ]:
# LSTM Model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=sequence_length, trainable=False))
model.add(LSTM(64))
model.add(Dense(vocab_size, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

# Model Training


In [ ]:
# Training
train_size = int(0.8 * len(input_sequences))

X_train = input_sequences[:train_size]
y_train = output_sequences[:train_size]
X_valid = input_sequences[train_size:]
y_valid = output_sequences[train_size:]

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=126, epochs=5,verbose=1)


Epoch 1/5
13134/13134 [==============================] - 122s 9ms/step - loss: 5.9144 - val_loss: 5.3207
Epoch 2/5
13134/13134 [==============================] - 111s 8ms/step - loss: 5.0689 - val_loss: 5.0657
Epoch 3/5
13134/13134 [==============================] - 104s 8ms/step - loss: 4.8319 - val_loss: 4.9679
Epoch 4/5
13134/13134 [==============================] - 112s 9ms/step - loss: 4.7061 - val_loss: 4.9226
Epoch 5/5
13134/13134 [==============================] - 112s 9ms/step - loss: 4.6261 - val_loss: 4.9000


In [ ]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=126, epochs=5,verbose=1)


Epoch 1/5
13134/13134 [==============================] - 111s 8ms/step - loss: 4.5678 - val_loss: 4.8848
Epoch 2/5
13134/13134 [==============================] - 111s 8ms/step - loss: 4.5265 - val_loss: 4.8788
Epoch 3/5
13134/13134 [==============================] - 111s 8ms/step - loss: 4.4951 - val_loss: 4.8799
Epoch 4/5
13134/13134 [==============================] - 111s 8ms/step - loss: 4.4691 - val_loss: 4.8769
Epoch 5/5
13134/13134 [==============================] - 111s 8ms/step - loss: 4.4480 - val_loss: 4.8763


In [ ]:
def predict_next_words(sentence, top_k=3):
    tokens = tokenizer.texts_to_sequences([sentence])[0]
    tokens = pad_sequences([tokens], maxlen=sequence_length)
    prediction = model.predict(tokens)[0]
    top_indices = prediction.argsort()[-top_k:][::-1]
    next_words = [tokenizer.index_word[idx] for idx in top_indices]
    return next_words

# Prediction Function


In [ ]:
def run_sentence_autocomplete():
    sentence = " "
    while True:
        word = input("Enter Next word (-1 to terminate): ")
        if word == "-1":
            break
        sentence += word + " "
        correct = "yes"
        while correct.lower() == "yes":
            next_words = predict_next_words(sentence)
            print("Top predicted words:", ", ".join(next_words))
            next_word = input("Choose the next word from the options above: ")
            if next_word in next_words:
                sentence += next_word + " "
            else:
                print("The word you chose is not in the top predictions. Please choose a word from the options.")
                correct = input("Is this the correct word? (yes/no): ")

    print("Your final sentence is:", sentence.strip())
    print()

In [ ]:
model.save('predict_next_word_model.keras')

In [ ]:
run_sentence_autocomplete()